<a href="https://colab.research.google.com/github/VVdovichev/ML_in_Business/blob/main/HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
#!pip install catboost

In [139]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier

##__1.__ взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [140]:
list_column_names = ['variance of Wavelet Transformed image',
                     'skewness of Wavelet Transformed image',
                     'curtosis of Wavelet Transformed image',
                     'entropy of image',
                     'class']
df = pd.read_csv('data_banknote_authentication.txt', names=list_column_names)
df.head(10)

,variance of Wavelet Transformed image,skewness of Wavelet Transformed image,curtosis of Wavelet Transformed image,entropy of image,class
0,3.62160,8.6661,-2.80730,-0.44699,0
1,4.54590,8.1674,-2.45860,-1.46210,0
2,3.86600,-2.6383,1.92420,0.10645,0
3,3.45660,9.5228,-4.01120,-3.59440,0
4,0.32924,-4.4552,4.57180,-0.98880,0
5,4.36840,9.6718,-3.96060,-3.16250,0
6,3.59120,3.0129,0.72888,0.56421,0
7,2.09220,-6.8100,8.46360,-0.60216,0
8,3.20320,5.7588,-0.75345,-0.61251,0
9,1.53560,9.1772,-2.27180,-0.73535,0


In [141]:
df.shape

(1372, 5)

In [142]:
df['class'].value_counts()

0    762
1    610
Name: class, dtype: int64

In [143]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.2, random_state=7)

##__2.__ сделать feature engineering

Этот шаг пропущу, т.к. буду использовать catboost, а ему не нужна обработка признаков.

##__3.__ бучить любой классификатор (какой вам нравится)

In [144]:
model = CatBoostClassifier(random_state=7)

In [145]:
model.fit(x_train, y_train)

Learning rate set to 0.010718
0:	learn: 0.6752852	total: 1.83ms	remaining: 1.83s
1:	learn: 0.6558661	total: 4.2ms	remaining: 2.1s
2:	learn: 0.6412052	total: 5.74ms	remaining: 1.91s
3:	learn: 0.6259161	total: 7.27ms	remaining: 1.81s
4:	learn: 0.6127214	total: 8.86ms	remaining: 1.76s
5:	learn: 0.5946619	total: 10.6ms	remaining: 1.75s
6:	learn: 0.5802288	total: 12.3ms	remaining: 1.75s
7:	learn: 0.5658303	total: 14.1ms	remaining: 1.75s
8:	learn: 0.5528240	total: 15.9ms	remaining: 1.75s
9:	learn: 0.5379378	total: 17.7ms	remaining: 1.75s
10:	learn: 0.5244803	total: 19.5ms	remaining: 1.75s
11:	learn: 0.5093472	total: 21.2ms	remaining: 1.75s
12:	learn: 0.4944736	total: 23ms	remaining: 1.75s
13:	learn: 0.4801284	total: 24.8ms	remaining: 1.75s
14:	learn: 0.4686450	total: 26.6ms	remaining: 1.74s
15:	learn: 0.4547835	total: 28.3ms	remaining: 1.74s
16:	learn: 0.4414699	total: 30ms	remaining: 1.73s
17:	learn: 0.4318382	total: 31.8ms	remaining: 1.73s
18:	learn: 0.4211892	total: 33.7ms	remaining: 1.74

In [146]:
pred = model.predict(x_test)
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, model.predict_proba(x_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics = pd.DataFrame(data={
    'threshold': list_threshold[index_best],
    'precision': list_precision[index_best],
    'recall': list_recall[index_best],
    'f1-score': list_f1_score[index_best]
}, index=['Base'])

df_metrics

,threshold,precision,recall,f1-score
Base,0.634229,0.991525,1.0,0.995745


##__4.__ далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [147]:
mod_df = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 45% of the positives marked
pos_sample_len = int(np.ceil(0.45 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 275/610 as positives and unlabeling the rest


In [148]:
mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    1097
 1     275
Name: class_test, dtype: int64


In [149]:
mod_df.head(10)

,variance of Wavelet Transformed image,skewness of Wavelet Transformed image,curtosis of Wavelet Transformed image,entropy of image,class,class_test
0,3.62160,8.6661,-2.80730,-0.44699,0,-1
1,4.54590,8.1674,-2.45860,-1.46210,0,-1
2,3.86600,-2.6383,1.92420,0.10645,0,-1
3,3.45660,9.5228,-4.01120,-3.59440,0,-1
4,0.32924,-4.4552,4.57180,-0.98880,0,-1
5,4.36840,9.6718,-3.96060,-3.16250,0,-1
6,3.59120,3.0129,0.72888,0.56421,0,-1
7,2.09220,-6.8100,8.46360,-0.60216,0,-1
8,3.20320,5.7588,-0.75345,-0.61251,0,-1
9,1.53560,9.1772,-2.27180,-0.73535,0,-1


In [150]:
x_data = mod_df.iloc[:,:-2].values # just the X 
y_labeled = mod_df.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_df.iloc[:,-2].values # original class

##__5.__ применить random negative sampling для построения классификатора в новых условиях

In [151]:
mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(275, 6) (275, 6)


In [152]:
model_mod = CatBoostClassifier(random_state=7)

model_mod.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model_mod.predict(sample_test.iloc[:,:-2].values)

Learning rate set to 0.007981
0:	learn: 0.6796651	total: 1.44ms	remaining: 1.44s
1:	learn: 0.6680991	total: 3.15ms	remaining: 1.57s
2:	learn: 0.6544917	total: 4.82ms	remaining: 1.6s
3:	learn: 0.6404427	total: 6.43ms	remaining: 1.6s
4:	learn: 0.6275340	total: 8.16ms	remaining: 1.62s
5:	learn: 0.6173568	total: 9.72ms	remaining: 1.61s
6:	learn: 0.6064611	total: 11.3ms	remaining: 1.6s
7:	learn: 0.5945398	total: 12.8ms	remaining: 1.59s
8:	learn: 0.5855906	total: 14.5ms	remaining: 1.6s
9:	learn: 0.5735881	total: 16.1ms	remaining: 1.6s
10:	learn: 0.5615816	total: 17.7ms	remaining: 1.59s
11:	learn: 0.5493632	total: 19.3ms	remaining: 1.59s
12:	learn: 0.5410677	total: 20.9ms	remaining: 1.59s
13:	learn: 0.5306573	total: 22.5ms	remaining: 1.58s
14:	learn: 0.5195479	total: 24.1ms	remaining: 1.58s
15:	learn: 0.5084032	total: 25.6ms	remaining: 1.58s
16:	learn: 0.4976293	total: 27.2ms	remaining: 1.57s
17:	learn: 0.4886950	total: 28.8ms	remaining: 1.57s
18:	learn: 0.4799633	total: 30.4ms	remaining: 1.5

##__6.__ сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [153]:
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, model_mod.predict_proba(x_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics.loc['Random Negative Sampling', :] = [
    list_threshold[index_best],
    list_precision[index_best],
    list_recall[index_best],
    list_f1_score[index_best]
]

df_metrics

,threshold,precision,recall,f1-score
Base,0.634229,0.991525,1.0,0.995745
Random Negative Sampling,0.936159,1.000000,1.0,1.000000


##__7.__ поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Уменьшение Positives

In [154]:
mod_df = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]
mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
x_data = mod_df.iloc[:,:-2].values # just the X 
y_labeled = mod_df.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_df.iloc[:,-2].values # original class
mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
model_mod = CatBoostClassifier(random_state=7)

model_mod.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model_mod.predict(sample_test.iloc[:,:-2].values)
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, model_mod.predict_proba(x_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics.loc['Random Negative Sampling Downgrade', :] = [
    list_threshold[index_best],
    list_precision[index_best],
    list_recall[index_best],
    list_f1_score[index_best]
]

df_metrics

Using 153/610 as positives and unlabeling the rest
(153, 6) (153, 6)
Learning rate set to 0.006214
0:	learn: 0.6848908	total: 1.65ms	remaining: 1.65s
1:	learn: 0.6762119	total: 3.29ms	remaining: 1.64s
2:	learn: 0.6699797	total: 4.88ms	remaining: 1.62s
3:	learn: 0.6606321	total: 6.45ms	remaining: 1.61s
4:	learn: 0.6532391	total: 8.04ms	remaining: 1.6s
5:	learn: 0.6458478	total: 10.1ms	remaining: 1.67s
6:	learn: 0.6382497	total: 11.9ms	remaining: 1.69s
7:	learn: 0.6290011	total: 13.5ms	remaining: 1.68s
8:	learn: 0.6218466	total: 15.1ms	remaining: 1.66s
9:	learn: 0.6150330	total: 16.7ms	remaining: 1.66s
10:	learn: 0.6069911	total: 18.1ms	remaining: 1.62s
11:	learn: 0.5996705	total: 19.6ms	remaining: 1.61s
12:	learn: 0.5933444	total: 21.1ms	remaining: 1.6s
13:	learn: 0.5851119	total: 22.7ms	remaining: 1.6s
14:	learn: 0.5764894	total: 24.2ms	remaining: 1.59s
15:	learn: 0.5685131	total: 25.9ms	remaining: 1.59s
16:	learn: 0.5605294	total: 27.4ms	remaining: 1.59s
17:	learn: 0.5552010	total: 29

,threshold,precision,recall,f1-score
Base,0.634229,0.991525,1.000000,0.995745
Random Negative Sampling,0.936159,1.000000,1.000000,1.000000
Random Negative Sampling Downgrade,0.921156,1.000000,0.991453,0.995708


In [155]:
mod_df = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 75% of the positives marked
pos_sample_len = int(np.ceil(0.75 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]
mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
x_data = mod_df.iloc[:,:-2].values # just the X 
y_labeled = mod_df.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_df.iloc[:,-2].values # original class
mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
model_mod = CatBoostClassifier(random_state=7)

model_mod.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model_mod.predict(sample_test.iloc[:,:-2].values)
list_precision, list_recall, list_threshold = precision_recall_curve(y_test, model_mod.predict_proba(x_test)[:, 1])

list_f1_score = 2 * (list_precision * list_recall) / (list_precision + list_recall)

index_best = np.argmax(list_f1_score)

df_metrics.loc['Random Negative Sampling Upgrade', :] = [
    list_threshold[index_best],
    list_precision[index_best],
    list_recall[index_best],
    list_f1_score[index_best]
]

df_metrics

Using 458/610 as positives and unlabeling the rest
(458, 6) (458, 6)
Learning rate set to 0.009923
0:	learn: 0.6764897	total: 1.65ms	remaining: 1.65s
1:	learn: 0.6608120	total: 3.21ms	remaining: 1.6s
2:	learn: 0.6458318	total: 4.71ms	remaining: 1.56s
3:	learn: 0.6300923	total: 6.22ms	remaining: 1.55s
4:	learn: 0.6171667	total: 8.87ms	remaining: 1.76s
5:	learn: 0.6060634	total: 11.4ms	remaining: 1.89s
6:	learn: 0.5917691	total: 15.1ms	remaining: 2.14s
7:	learn: 0.5774455	total: 17ms	remaining: 2.11s
8:	learn: 0.5649951	total: 18.7ms	remaining: 2.06s
9:	learn: 0.5537068	total: 20.5ms	remaining: 2.03s
10:	learn: 0.5413967	total: 22.7ms	remaining: 2.04s
11:	learn: 0.5286180	total: 24.4ms	remaining: 2s
12:	learn: 0.5146576	total: 26ms	remaining: 1.98s
13:	learn: 0.5018758	total: 27.8ms	remaining: 1.96s
14:	learn: 0.4894331	total: 29.5ms	remaining: 1.94s
15:	learn: 0.4773060	total: 31.2ms	remaining: 1.92s
16:	learn: 0.4659951	total: 32.9ms	remaining: 1.9s
17:	learn: 0.4593402	total: 34.7ms	r

,threshold,precision,recall,f1-score
Base,0.634229,0.991525,1.000000,0.995745
Random Negative Sampling,0.936159,1.000000,1.000000,1.000000
Random Negative Sampling Downgrade,0.921156,1.000000,0.991453,0.995708
Random Negative Sampling Upgrade,0.954418,1.000000,1.000000,1.000000
